## 11.3 Develop a Grid Search Framework

In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings

In [3]:
# one step naive forecast
def naive_forecast(history, n):
    return(history[-n])

# define dataset
dataset = np.arange(10, 110, 10)
print(dataset)
#
for i in range(1, dataset.shape[0]+1):
    print(naive_forecast(dataset, i))

[ 10  20  30  40  50  60  70  80  90 100]
100
90
80
70
60
50
40
30
20
10


The average forecast() function below implements this taking the historical data and a
config array or tuple that specifies the number of prior values to average as an integer, and a string that describe the way to calculate the average (mean or median)

In [4]:
def average_forecast(history, config):
    n, avg_type = config
    print('## history = {}'.format(history[-n:]))
    # mean
    if avg_type == 'mean':
        return(np.average(history[-n:]))
    if avg_type == 'median':
        return(np.median(history[-n:]))

print(dataset.shape[0])
for i in range(1, dataset.shape[0]+1):
    print(i, average_forecast(dataset, (i, 'mean')))

10
## history = [100]
1 100.0
## history = [ 90 100]
2 95.0
## history = [ 80  90 100]
3 90.0
## history = [ 70  80  90 100]
4 85.0
## history = [ 60  70  80  90 100]
5 80.0
## history = [ 50  60  70  80  90 100]
6 75.0
## history = [ 40  50  60  70  80  90 100]
7 70.0
## history = [ 30  40  50  60  70  80  90 100]
8 65.0
## history = [ 20  30  40  50  60  70  80  90 100]
9 60.0
## history = [ 10  20  30  40  50  60  70  80  90 100]
10 55.0


In [5]:
# one step average forecast
def average_forecast1(history, config):
    n, offset, avg_type = config
    values = list()
    if offset == 1:
        values = history[-n:]
    else:
        # skip bad config
        if n*offset > history.shape[0]:
            raise Exception('Config beyond end of data: {} {}'.format(n,offset))
        # try and collect n values using offset
        for i in range(1, n+1):
            ix = i * offset
            print('negative position : {}'.format(-ix))
            values.append(history[-ix])
    # mean of last n values
    print('{} component values = {}'.format(avg_type, values))
    if avg_type == 'mean':
        return np.mean(values)
    if avg_type == 'median':
        return np.median(values)
    
#
data = np.array([10, 20, 30, 10, 20, 30, 10, 20, 30])
print(data)
# test naive forecast
for i in [1, 2, 3]:
    # i = number of elements picked from the dataset
    print(average_forecast1(data, (i, 2, 'mean')))

[10 20 30 10 20 30 10 20 30]
negative position : -2
mean component values = [20]
20.0
negative position : -2
negative position : -4
mean component values = [20, 30]
25.0
negative position : -2
negative position : -4
negative position : -6
mean component values = [20, 30, 10]
20.0


- It is possible to combine both the naive and the average forecast strategies together into
the same function.
- There is a little overlap between the methods, specifically the n-offset into the history that is used to either persist values or determine the number of values to average.

In [6]:
# one step average forecast
def simple_forecast1(history, config):
    n, offset, avg_type = config
    # if persist value, ignore other config
    if avg_type == 'persist':
        return history[-n]
    # collect values to average
    values = list()
    if offset == 1:
        values = history[-n:]
    else:
        # skip bad config
        if n*offset > history.shape[0]:
            raise Exception('Config beyond end of data: {} {}'.format(n,offset))
        # try and collect n values using offset
        for i in range(1, n+1):
            ix = i * offset
            print('negative position : {}'.format(-ix))
            values.append(history[-ix])
    # mean of last n values
    print('{} subprocess values = {}'.format(avg_type, values))
    if avg_type == 'mean':
        return np.mean(values)
    return np.median(values)

def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

def measure_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

In [7]:
dataset

array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100])

In [8]:
print(dataset[:-3], dataset[-3:])

[10 20 30 40 50 60 70] [ 80  90 100]


- We can now implement the walk-forward validation scheme. 
- This is a standard approach to evaluating a time series forecasting model that respects the temporal ordering of observations. 
- First, a provided univariate time series dataset is split into train and test sets using the train test split() function. 
- Then the number of observations in the test set are enumerated. 
- For each we fit a model on all of the history and make a one step forecast. 
- The true observation for the time step is then added to the history, and the process is repeated.
- The simple forecast() function is called in order to fit a model and make a prediction. 
- Finally, an error score is calculated by comparing all one-step forecasts to the actual test set by calling the measure rmse() function. 
- The walk forward validation() function below implements this, taking a univariate time series, a number of time steps to use in the test set, and an array of model configuration.

In [9]:
def walk_forward_validation(data, n_test, cfg):
    prediction = list()
    # split dataset
    X_train, X_test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in X_train]
    # step over each time stpe in the test set
    for i in range(len(X_test)):
        # fit model and make forecast for history
        yhat = simple_forecast1(history, cfg)
        # fit model and make forecast for history
        prediction.append(yhat)
        # add actual observation to history for the next loop
        history.append(X_test[i])
    # estimate prediction error
    error = measure_rmse(X_test, prediction)
    return error

def score_model(data, n_test, cfg, debug=True):
    result = None
    # convert config to key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings('ignore')
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    if result is not None:
        print(' > Model {} rmse error {}'.format(key, result))
    return (key, result)

In [32]:
# create a set of simple configs to try
def simple_configs(max_length, offsets=[1]):
    configs = list()
    for i in range(1, max_length+1):
        for o in offsets:
            for t in ['persist', 'mean', 'median']:
                cfg = [i, o, t]
                configs.append(cfg)
    return configs

def grid_search(data, cfg_list, n_test, parallel=False):
    scores = None
    if parallel:
        cpu = cpu_count()
        cpu = 3
        print('cpu_count {}'.format(cpu))
        executor = Parallel(n_jobs=cpu, backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [s for s in scores if s != None]
    # sort scores
    scores.sort(key=lambda tup: tup[1])
    return scores

In [22]:
data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
n_test = 4
max_length = len(data) - n_test
cfg_list = simple_configs(max_length)
print(cfg_list)

[[1, 1, 'persist'], [1, 1, 'mean'], [1, 1, 'median'], [2, 1, 'persist'], [2, 1, 'mean'], [2, 1, 'median'], [3, 1, 'persist'], [3, 1, 'mean'], [3, 1, 'median'], [4, 1, 'persist'], [4, 1, 'mean'], [4, 1, 'median'], [5, 1, 'persist'], [5, 1, 'mean'], [5, 1, 'median'], [6, 1, 'persist'], [6, 1, 'mean'], [6, 1, 'median']]


In [23]:
%time scores = grid_search(data, cfg_list, n_test, parallel=False)
print(' done' ) # list top 3 configs 
for cfg, error in scores[:3]:
    print(cfg, error)

 > Model [1, 1, 'persist'] rmse error 10.0
mean subprocess values = [60.0]
mean subprocess values = [70.0]
mean subprocess values = [80.0]
mean subprocess values = [90.0]
 > Model [1, 1, 'mean'] rmse error 10.0
median subprocess values = [60.0]
median subprocess values = [70.0]
median subprocess values = [80.0]
median subprocess values = [90.0]
 > Model [1, 1, 'median'] rmse error 10.0
 > Model [2, 1, 'persist'] rmse error 20.0
mean subprocess values = [50.0, 60.0]
mean subprocess values = [60.0, 70.0]
mean subprocess values = [70.0, 80.0]
mean subprocess values = [80.0, 90.0]
 > Model [2, 1, 'mean'] rmse error 15.0
median subprocess values = [50.0, 60.0]
median subprocess values = [60.0, 70.0]
median subprocess values = [70.0, 80.0]
median subprocess values = [80.0, 90.0]
 > Model [2, 1, 'median'] rmse error 15.0
 > Model [3, 1, 'persist'] rmse error 30.0
mean subprocess values = [40.0, 50.0, 60.0]
mean subprocess values = [50.0, 60.0, 70.0]
mean subprocess values = [60.0, 70.0, 80.0]

In [33]:
%time scores = grid_search(data, cfg_list, n_test, parallel=True)
print(' done' ) # list top 3 configs 
for cfg, error in scores[:3]:
    print(cfg, error)

cpu_count 3
mean subprocess values = [60.0]
 > Model [1, 1, 'persist'] rmse error 10.0
median subprocess values = [60.0]
mean subprocess values = [70.0]
median subprocess values = [70.0]
mean subprocess values = [80.0]
median subprocess values = [80.0]
 > Model [2, 1, 'persist'] rmse error 20.0
mean subprocess values = [90.0]
median subprocess values = [90.0]
mean subprocess values = [50.0, 60.0]
 > Model [1, 1, 'mean'] rmse error 10.0
 > Model [1, 1, 'median'] rmse error 10.0
mean subprocess values = [60.0, 70.0]
median subprocess values = [50.0, 60.0]
 > Model [3, 1, 'persist'] rmse error 30.0
mean subprocess values = [70.0, 80.0]
mean subprocess values = [40.0, 50.0, 60.0]
median subprocess values = [60.0, 70.0]
mean subprocess values = [80.0, 90.0]
mean subprocess values = [50.0, 60.0, 70.0]
median subprocess values = [70.0, 80.0]
 > Model [2, 1, 'mean'] rmse error 15.0
mean subprocess values = [60.0, 70.0, 80.0]
median subprocess values = [80.0, 90.0]
mean subprocess values = [70.